<a href="https://colab.research.google.com/github/dipnarayan501/Assignment3/blob/main/seq2seqvanilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Seq2Seq model**


Mounted the drive and unzip the datset (hindi)

In [ ]:
#Mount to google drive
from google.colab import drive
drive.mount('/content/drive')

#Unziping dataset
zip_path = "drive/MyDrive/hi.zip"
!cp "{zip_path}" .
!unzip -q hi.zip

Mounted at /content/drive


configuring wandb

In [ ]:
#import libraries
%pip install wandb -q
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 32.0 MB/s 
     |████████████████████████████████| 144 kB 59.7 MB/s 
     |████████████████████████████████| 181 kB 45.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

import libraries

In [ ]:
#Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import SimpleRNN,LSTM,GRU,Embedding,Dense,Dropout,Input
from tensorflow.keras.optimizers import Adam,Nadam
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle
import keras
import tensorflow as tf
from wandb.keras import WandbCallback

**Loading** dataset 

In [ ]:
#Reading dataset as dataframe and returning it
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["hi","en",""],skip_blank_lines=True,index_col=None)
    data = data[data['hi'].notna()]
    data = data[data['en'].notna()]
    data = data[['hi','en']]
    return data

Getting dataset for train test validation 

In [ ]:
train = load_data("/content/hi/lexicons/hi.translit.sampled.train.tsv")
dev = load_data("/content/hi/lexicons/hi.translit.sampled.dev.tsv")
test = load_data("/content/hi/lexicons/hi.translit.sampled.test.tsv")

Getting Unique tokens hindi and english language

In [ ]:
def unique_tokenize(data):
    english = train['en'].values
    hindi = train['hi'].values
    hindi = '\t'+hindi+'\n'
    english_tokens = set()
    hindi_tokens = set()
    
    for x,y in zip(english,hindi):
        for ch in x:
            english_tokens.add(ch)
        for ch in y:
            hindi_tokens.add(ch)
    english_tokens = sorted(list(english_tokens))
    hindi_tokens = sorted(list(hindi_tokens))
    return hindi_tokens , english_tokens
hindi_tokens , english_tokens = unique_tokenize(train)

Mapping the tokens for hindi and engliish

In [ ]:
def tokenize_map(hindi_tokens , english_tokens):
    english_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
    hindi_token_map = dict([(ch,i+1) for i,ch in enumerate(hindi_tokens)])
    #Adding blank space
    hindi_token_map[" "] = 0
    english_token_map[" "] = 0
    return hindi_token_map, english_token_map

hin_token_map, eng_token_map = tokenize_map(hindi_tokens , english_tokens)

Getting maximum length 

In [ ]:
x = train['en'].values
y = train['hi'].values
y = '\t'+y+'\n'

#Getting max length
max_eng_len = max([len(i) for i in x])
max_hin_len = max([len(i) for i in y])

Preprocessing the datset 

In [ ]:
def process(data):
    x,y = data['en'].values, data['hi'].values
    y = "\t" + y + "\n"
    
    a = np.zeros((len(x),max_eng_len),dtype="float32")
    b = np.zeros((len(y),max_hin_len),dtype="float32")
    c = np.zeros((len(y),max_hin_len,len(hindi_tokens)+1),dtype="int")
    
    
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = eng_token_map[ch]

        a[i,j+1:] = eng_token_map[" "]
        for j,ch in enumerate(yy):
            b[i,j] = hin_token_map[ch]

            if j>0:
                c[i,j-1,hin_token_map[ch]] = 1

        b[i,j+1:] = hin_token_map[" "]
        c[i,j:,hin_token_map[" "]] = 1
        
    return a,b,c

In [ ]:
#Getting preprocess train test and validation data
trainx, trainxx, trainy = process(train)
valx, valxx, valy = process(dev)
testx,testxx,testy = process(test)
np.random.seed(42)
reverse_eng_map = dict([(i,char) for char,i in eng_token_map.items()])
reverse_hin_map = dict([(i,char) for char,i in hin_token_map.items()])

Generating encoder decoder for LSTM , RNN, GRU

In [ ]:
def build_model(cell = "LSTM",units = 32, enc_layers = 1, dec_layers = 1,embedding_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=len(english_tokens)+1,output_dim = embedding_dim,mask_zero=True)
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim = len(hindi_tokens)+1,output_dim = embedding_dim,mask_zero=True)
    decoder_context = decoder_embedding(decoder_inputs)
  
    if cell == "LSTM":
        encoder_prev = [LSTM(units,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = LSTM(units,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        _, state_h,state_c = encoder_fin(temp)
        encoder_states = [state_h,state_c]
        
        decoder = [LSTM(units,return_sequences=True,return_state=True) for i in range(dec_layers)]
        
        temp,sh,sc = decoder[0](decoder_context,initial_state=encoder_states)
        for i in range(1,dec_layers):
            temp,sh,sc = decoder[i](temp,initial_state=encoder_states)

    elif cell == "GRU":
        encoder_prev = [GRU(units,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = GRU(units,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        _, state = encoder_fin(temp)
        encoder_states = state
        
        decoder = [GRU(units,return_sequences=True,return_state=True) for i in range(dec_layers)]
        
        temp,s = decoder[0](decoder_context,initial_state=state)
        for i in range(1,dec_layers):
            temp,s = decoder[i](temp,initial_state=state)

    elif cell == "RNN":
        encoder_prev = [SimpleRNN(units,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = SimpleRNN(units,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        _, state = encoder_fin(temp)
        encoder_states = state
        
        decoder = [SimpleRNN(units,return_sequences=True,return_state=True) for i in range(dec_layers)]
        
        temp,s = decoder[0](decoder_context,initial_state=state)
        for i in range(1,dec_layers):
            temp,s = decoder[i](temp,initial_state=state)
       

            
        
    dense_lay1 = Dense(dense_size,activation='relu')
    pre_out = dense_lay1(temp)
    dense_lay2 = Dense(len(hindi_tokens)+1,activation = 'softmax')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    encoder_model = Model(encoder_inputs,encoder_states)
    
    if cell == "LSTM":
        state_inputs = []
        state_outputs = []
        
        decoder_input_h = Input(shape=(units,))
        decoder_input_c = Input(shape=(units,))
        temp,sh,sc = decoder[0](decoder_context,initial_state = [decoder_input_h,decoder_input_c])
        state_inputs += [decoder_input_h,decoder_input_c]
        state_outputs += [sh,sc]
                                                                  
        for i in range(1,dec_layers):
            decoder_input_h = Input(shape=(units,))
            decoder_input_c = Input(shape=(units,))
            temp,sh,sc = decoder[i](temp,initial_state = [decoder_input_h,decoder_input_c])
            state_inputs += [decoder_input_h,decoder_input_c]
            state_outputs += [sh,sc]
            
        decoder_input_pass = [decoder_inputs] + state_inputs
    elif cell == "GRU":
        state_inputs = []
        state_outputs = []
        
        state_input = Input(shape=(units,))
        temp,s = decoder[0](decoder_context,initial_state = state_input)
        state_inputs.append(state_input)
        state_outputs.append(s)
                                                                  
        for i in range(1,dec_layers):
            state_input = Input(shape=(units,))
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)
            
        decoder_input_pass = [decoder_inputs] + state_inputs

    elif cell == "RNN":
        state_inputs = []
        state_outputs = []
        
        state_input = Input(shape=(units,))
        temp,s = decoder[0](decoder_context,initial_state = state_input)
        state_inputs.append(state_input)
        state_outputs.append(s)
                                                                  
        for i in range(1,dec_layers):
            state_input = Input(shape=(units,))
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)
            
        decoder_input_pass = [decoder_inputs] + state_inputs
        


    pre_out = dense_lay1(temp)
    final_output = dense_lay2(pre_out)
    
    decoder_model = Model(decoder_input_pass, [final_output]+state_outputs)
    
    return train,encoder_model,decoder_model

configuring wandb

In [ ]:

import wandb
from wandb.keras import WandbCallback
%pip install wandb -q
import wandb
wandb.login()


wandb: Currently logged in as: dipnarayan501 (fdl-moni_dip). Use `wandb login --relogin` to force relogin


True

In [ ]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'learning_rate': 1e-2,
        'dense_size': 128,
        'units': 128,
        'cell': 'LSTM',
        'embedding_dim': 64,
        'enc_layers': 1,
        'dec_layers': 1,
        'dropout': 0.,
        'batch_size': 64
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    
    dense_size=config.dense_size
    cell = config.cell
    dropout = config.dropout
    enc_layers=config.enc_layers
    dec_layers=config.dec_layers
    embedding_dim = config.embedding_dim
    units = config.units

    learning_rate = config.learning_rate
    batch_size = config.batch_size
                                

    # Displaying hyperparameters
    run_name = "cell_{}_enc_lay_{}_dec_lay_{}_units_{}_embd_{}_dp_{}_lr_{}_ds_{}_bs_{}".format(cell, enc_layers, dec_layers,units,embedding_dim, dropout, learning_rate, dense_size,batch_size)
    print(run_name)    
    # Config is a variable
    train,enc,dec = build_model(units=units,
                                dense_size=dense_size,
                                enc_layers=enc_layers,
                                dec_layers=dec_layers,
                                cell = cell,
                                dropout = dropout,
                                embedding_dim = embedding_dim)
    train.compile(optimizer = Adam(learning_rate= learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])
    
    # Early Stopping 
    earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

    # To save the model with best validation accuracy
    checkpoint = ModelCheckpoint('bestmodel.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
    train.fit([trainx,trainxx],trainy,
             batch_size=batch_size,
             validation_data = ([valx,valxx],valy),
             epochs=10,
             callbacks=[WandbCallback(), earlyStopping,checkpoint])

    print("model training done")
    wandb.run.name = run_name
    wandb.run.save()
    return train

Sweep configuration 

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'learning_rate': {
            'values': [0.01, 0.001]
        },
        'dense_size': {
            'values': [64,128,512]
        },
        'dropout': {
            'values': [0.0,0.2,0.4]
        },
        'units': {
            'values': [64,128,256]
        },
          'batch_size': {
            'values': [64,128,256]
        },
        'cell': {
            'values': ["LSTM","GRU","RNN"]
        },
        'embedding_size': {
            'values': [64,128,256]
        },
        'enc_layers': {
            'values': [1,2,3]
        },
        'dec_layers': {
            'values': [1,2,3]
        },
        
    }
}

#sweep_id = wandb.sweep(sweep_config, entity="fdl-moni_dip", project="s2s_vannila")

sweep id 

In [ ]:
id = '152q0uoh'
wandb.agent(id, train,entity="fdl-moni_dip", project="s2s_vannila", count=1)

wandb: Agent Starting Run: gsx5sp73 with config:
wandb: 	batch_size: 128
wandb: 	cell: RNN
wandb: 	dec_layers: 1
wandb: 	dense_size: 128
wandb: 	dropout: 0.2
wandb: 	embedding_size: 256
wandb: 	enc_layers: 2
wandb: 	learning_rate: 0.001
wandb: 	units: 256


cell_RNN_enc_lay_2_dec_lay_1_units_256_embd_64_dp_0.2_lr_0.001_ds_128_bs_128


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
346/346 [==============================] - 108s 297ms/step - loss: 0.9009 - accuracy: 0.3970 - val_loss: 0.6746 - val_accuracy: 0.5140 - _timestamp: 1651644979.0000 - _runtime: 114.0000
Epoch 2/10
346/346 [==============================] - 88s 254ms/step - loss: 0.5906 - accuracy: 0.5724 - val_loss: 0.4699 - val_accuracy: 0.6310 - _timestamp: 1651645067.0000 - _runtime: 202.0000
Epoch 3/10
346/346 [==============================] - 75s 215ms/step - loss: 0.4367 - accuracy: 0.6656 - val_loss: 0.3799 - val_accuracy: 0.6984 - _timestamp: 1651645141.0000 - _runtime: 276.0000
Epoch 4/10
346/346 [==============================] - 80s 232ms/step - loss: 0.3657 - accuracy: 0.7136 - val_loss: 0.3376 - val_accuracy: 0.7263 - _timestamp: 1651645222.0000 - _runtime: 357.0000
Epoch 5/10
346/346 [==============================] - 74s 215ms/step - loss: 0.3230 - accuracy: 0.7448 - val_loss: 0.3111 - val_accuracy: 0.7456 - _timestamp: 1651645296.0000 - _runtime: 431.0000
Epoch 6/10
346/346 

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


model training done


accuracy,▁▄▆▆▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▂▂▂▁▁▁▁
val_accuracy,▁▄▆▇▇▇████
val_loss,█▄▃▂▂▂▁▁▁▁
accuracy,0.81139
best_epoch,9
best_val_loss,0.26711
epoch,9
loss,0.23244
val_accuracy,0.7811


Retraining model with best Parameters 


In [ ]:
train,enc,dec = build_model(units=256,dense_size=512,enc_layers=2,dec_layers=3,cell = "GRU", embedding_dim = 64)
train.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5',monitor='val_accuracy',mode='max',save_best_only=True,verbose=1)
train.fit([trainx,trainxx],trainy,
         batch_size=128,
         validation_data=([valx,valxx],valy),
         epochs=10,
          callbacks = [checkpoint])

Epoch 1/10
346/346 [==============================] - ETA: 0s - loss: 0.9571 - accuracy: 0.3476
Epoch 1: val_accuracy improved from -inf to 0.49960, saving model to best_model.h5
346/346 [==============================] - 320s 867ms/step - loss: 0.9571 - accuracy: 0.3476 - val_loss: 0.6602 - val_accuracy: 0.4996
Epoch 2/10
346/346 [==============================] - ETA: 0s - loss: 0.4859 - accuracy: 0.6361
Epoch 2: val_accuracy improved from 0.49960 to 0.73006, saving model to best_model.h5
346/346 [==============================] - 289s 835ms/step - loss: 0.4859 - accuracy: 0.6361 - val_loss: 0.3358 - val_accuracy: 0.7301
Epoch 3/10
346/346 [==============================] - ETA: 0s - loss: 0.2816 - accuracy: 0.7791
Epoch 3: val_accuracy improved from 0.73006 to 0.80314, saving model to best_model.h5
346/346 [==============================] - 299s 864ms/step - loss: 0.2816 - accuracy: 0.7791 - val_loss: 0.2399 - val_accuracy: 0.8031
Epoch 4/10
346/346 [==============================] 

In [ ]:
#Defining inference model and getting prediction
def inference(inp,dec_layers,cell="LSTM"):
    statess = enc.predict(inp)
    target_seq = np.zeros((inp.shape[0],1))
    target_seq[:,0] = hin_token_map["\t"]
    
    states = []
    
    if cell == "LSTM":
        for c in range(dec_layers):
            states += [statess[0],statess[1]]
            
    else:
        for c in range(dec_layers):
            states += [statess]
            
    ans = np.zeros((inp.shape[0],max_hin_len))
    
    for i in range(max_hin_len):
        output = dec.predict([target_seq]+states,batch_size=64)
        ans[:,i] = np.argmax(output[0][:,-1,:],axis=1)
        target_seq[:,0] = ans[:,i]
        states = output[1:]
        
    return ans

In [ ]:

#Predicting test word
prediction = inference(testx,3,cell="GRU")

#printing 10 sample outputs
for i in range(10):
    idx = np.random.choice(testx.shape[0])
    orig = ""
    for ch in testx[idx]:
        orig += reverse_eng_map[ch]
        if reverse_eng_map[ch] == "\n":
            break
        
    deco = ""
    for ch in prediction[idx]:
        deco += reverse_hin_map[ch]
        if reverse_hin_map[ch] == "\n":
            break
        
    print("Input :",orig)
    print("Output:", deco)
    print("********")

Input : kajoo               
Output: काजू

********
Input : purjon              
Output: पूर्जों

********
Input : peratrupers         
Output: पर्टरपर्स

********
Input : chaupal             
Output: चौपाल

********
Input : raak                
Output: राक

********
Input : saleeb              
Output: सलीब

********
Input : borivali            
Output: बोरीवाली

********
Input : untees              
Output: अन्टीस

********
Input : chand               
Output: चांड

********
Input : farukhnagar         
Output: फरुक्चानगार

********


In [ ]:
#Getting output words
output_words = []
for i in range(testx.shape[0]):
    idx = i    
    decode = ""
    for ch in prediction[idx]:
        decode += reverse_hin_map[ch]
        if reverse_hin_map[ch] == "\n":
            break

    output_words.append(decode)

print(len(output_words))


4502


In [ ]:
#Saving ouput as prediction_vanilla,csv from dataframe
pred = pd.DataFrame(output_words)
pred1 = pred.replace('\\n','', regex=True)
final_pred = pred1.replace('\\t','', regex=True)


test['pred_vanilla'] = final_pred.values
test.rename(columns = {'hi':'target', 'en':'input'}, inplace = True)
test.to_csv('prediction_vanilla.csv', sep='\t', encoding='utf-8')


In [ ]:
test.head(15)

,target,input,pred_vanilla
0,अंक,ank,अंक
1,अंक,anka,अंका
2,अंकित,ankit,अंकित
3,अंकों,anakon,अनाकों
4,अंकों,ankhon,अंखों
5,अंकों,ankon,अंकों
6,अंकोर,angkor,अंगकोर
7,अंकोर,ankor,अंकर
8,अंगारक,angaarak,अंगारक
9,अंगारक,angarak,अंगारक


In [ ]:
#Test accuracy by matching exact string
def test_accuracy(pred): 
  acc = 0

  for i,pr in enumerate(pred):
      fl = 1
      for j,ch in enumerate(pr):
          if ch != np.argmax(testy[i,j,:]):
              fl = 0
              break
          if ch == hin_token_map["\n"]:
              break
              
      if fl==1:
          acc+=1
    
  return (acc/len(pred))*100



print("Test accuracy with best parameters of s2s_without attention  ", test_accuracy(prediction))

Test accuracy with best parameters of s2s_without attention   34.606841403820525
